# loop

在编译原理课程项目中，进行自然循环识别的代码。

## 需要的库

- 正则表达式 re
- 序列化 json

In [1]:
import re
import json
import copy

## 读入json

In [41]:
filename = "../examples/vm_src04_blk.json"
vm_blk = {}
with open(filename, "r") as fp:
    vm_blk = json.load(fp)
blocks = vm_blk["blocks"]

cond_f = {"==": lambda x, y: x == y, "!=": lambda x, y: x != y, ">=": lambda x, y: x >=
          y, "<=": lambda x, y: x <= y, ">": lambda x, y: x > y, "<": lambda x, y: x < y}
arith_f = {"+": lambda x, y: x+y, "-": lambda x, y: x-y, "*": lambda x,
           y: x*y, "/": lambda x, y: x//y, "%": lambda x, y: x % y}
eserved_word = {"HALT", "=", "+", "-", "*", "/", "%", "?",
                ":", "!:", ">", "<", ">=", "<=", "==", "!=", "[", "]"}
reg = re.compile('^[0-9]+$')

In [42]:
blocks

{'0': {'line_num': [0, 0],
  'next': [1, None],
  'code': ['B0 = 0'],
  'defd': ['B0'],
  'used': [],
  'in': ['B2', 'B4'],
  'out': ['B0', 'B2', 'B4']},
 '1': {'line_num': [1, 2],
  'next': [2, 6],
  'code': ['B1 = 1', '? B1 > B0 : 11'],
  'defd': ['B1'],
  'used': ['B0'],
  'in': ['B2', 'B0', 'B4'],
  'out': ['B0', 'B2', 'B4']},
 '2': {'line_num': [3, 4],
  'next': [4, None],
  'code': ['B2 = 2', '!: 7'],
  'defd': ['B2'],
  'used': [],
  'in': ['B0', 'B4'],
  'out': ['B0', 'B2', 'B4']},
 '3': {'line_num': [5, 6],
  'next': [4, None],
  'code': ['T0 = 233', 'T1 = 233'],
  'defd': ['T1', 'T0'],
  'used': [],
  'in': ['B2', 'B0', 'B4'],
  'out': ['B0', 'B2', 'B4']},
 '4': {'line_num': [7, 8],
  'next': [5, 1],
  'code': ['B3 = 3', '? B3 > B2 : 1'],
  'defd': ['B3'],
  'used': ['B2'],
  'in': ['B0', 'B2', 'B4'],
  'out': ['B0', 'B2', 'B4']},
 '5': {'line_num': [9, 10],
  'next': [None, None],
  'code': ['B4 = 4', 'HALT'],
  'defd': ['B4'],
  'used': [],
  'in': [],
  'out': []},
 '6': {

# 给基本块加入前驱节点属性

In [46]:
for n,b in blocks.items():
    blocks[n]["pre"] = set()

for n,b in blocks.items():
    for i in b["next"]:
        if i!=None:
            blocks[str(i)]["pre"] = blocks[str(i)]["pre"] | {n}
# blocks['0']["pre"] = {'E'}
for n,b in blocks.items():
    # blocks[n]["pre"] = list(blocks[n]["pre"])
    print(n,b["pre"])

0 set()
1 {'0', '4'}
2 {'1'}
3 set()
4 {'8', '3', '2'}
5 {'4'}
6 {'1'}
7 {'6'}
8 {'9', '7'}
9 {'6'}


# 构建支配关系

In [50]:
for n,b in blocks.items():
    blocks[n]["dom"] = set()

for n,b in blocks.items():
    blocks[n]["dom"] = set([str(i) for i in range(vm_blk["summary"]["total_blocks"])])
blocks[str(0)]["dom"] = {'0'}

for n,b in blocks.items():
    print(n,b["dom"])

flag = True
i = 0
while flag:
    flag = False
    i+=1
    print("#",i)
    for n,b in blocks.items():
        if n!='0':
            domset_ori = copy.deepcopy(blocks[n]["dom"])
            domset = copy.deepcopy(blocks[n]["dom"])
            preset = set([str(i) for i in range(vm_blk["summary"]["total_blocks"])])
            for j in blocks[n]["pre"]:
                preset = preset & blocks[j]["dom"]
            domset = {n} | preset
            if (domset!=domset_ori):
                flag = True
            blocks[n]["dom"] = domset
    for n,b in blocks.items():
        print(n,b["dom"])

0 {'0'}
1 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
2 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
3 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
4 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
5 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
6 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
7 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
8 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
9 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
# 1
0 {'0'}
1 {'1', '0'}
2 {'1', '2', '0'}
3 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
4 {'1', '2', '0', '4'}
5 {'1', '2', '4', '0', '5'}
6 {'1', '6', '0'}
7 {'1', '6', '7', '0'}
8 {'1', '6', '8', '7', '0'}
9 {'9', '6', '0', '1'}
# 2
0 {'0'}
1 {'1', '0'}
2 {'1', '2', '0'}
3 {'9', '8', '3', '0', '1', '6', '2', '4', '7', '5'}
4 {'1', '0', '4'}
5 {'1', '0', '5', '4'}
6 {'1', '6', '0'}
7 {'1', '6', '7', '0'}
8 {'1', '6', '8', '0'}
9 {'9', '6', '0', '1'}
# 3
0 {'0'}
1 {'1', '0'}
2 {'1', '2', '0'}
3 {'9', '